Using publicly available data at data.gov.sg please create a model pipeline to forecast (upcoming 3 hours) the traffic flow at the specified location (latitude: 1.357098686 longitude: 103.902042), for a specified time of day. The solution must have the following components:

1. Estimation of historical traffic flow, using image data sets available here (https://data.gov.sg/dataset/traffic-images )

2. Use the pipeline from (1) and weather data (https://data.gov.sg/dataset/realtime-weather-readings), to forecast the traffic flow at the specified location at a specified time of day

Notes: You may use any additional data sources / APIs to build your models 

In [1]:
!pip install cvlib
!pip install pystan==2.19.1.1
!pip install prophet
!pip install opencv-python==4.1.2.30
!pip install openpyxl

^C
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.62
    Uninstalling opencv-python-4.5.5.62:
      Successfully uninstalled opencv-python-4.5.5.62


In [2]:
# Import Libraries
import requests, json, joblib
import sys
import os
import pandas as pd, numpy as np
from datetime import datetime, timedelta
import pytz
import urllib
import cv2
from PIL import Image
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
# from prophet import Prophet

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Interviews/Siemens/data/')
print("All the system paths:")
sys.path

All the system paths:


['C:\\Users\\yeewl\\repo\\TrafficFlowPredictor',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\python37.zip',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\DLLs',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\lib',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS',
 '',
 'C:\\Users\\yeewl\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\yeewl\\AppData\\Roaming\\Python\\Python37\\site-packages\\win32',
 'C:\\Users\\yeewl\\AppData\\Roaming\\Python\\Python37\\site-packages\\win32\\lib',
 'C:\\Users\\yeewl\\AppData\\Roaming\\Python\\Python37\\site-packages\\Pythonwin',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\lib\\site-packages',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\lib\\site-packages\\win32',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\yeewl\\.conda\\envs\\ISS\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\yeewl\\AppData\\Roaming\\Python\\Python37\\site-packages\\IPython\\extensions',
 'C:\\Users\\yeewl\\.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/Interv

In [5]:
os.chdir('./data/')
print("Current working directory:")
os.getcwd()

Current working directory:


'C:\\Users\\yeewl\\repo\\TrafficFlowPredictor\\data'

## Define `cv2plt` for image display

In [6]:
def cv2plt(img):
    plt.figure(figsize=(8,8))        # To change the size of figure
    plt.axis('off')
    if np.size(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img,cmap='gray',vmin=0,vmax=255)  
    plt.show()

# Using YOLO v3

In [7]:
lbl_file        = 'yolov3.txt'
classes         = open(lbl_file).read().strip().split("\n")

                                                # Read in the deep learning net
yoloconfig      = 'yolov3.cfg'
yoloweights     = 'yolov3.weights'
net             = cv2.dnn.readNet(yoloweights,yoloconfig)

In [8]:
def getOutputLayers(net):
    layers = net.getLayerNames()
    outLayers = [layers[i[0]-1] for i in net.getUnconnectedOutLayers()]

    return outLayers


def yoloV3Detect(img,scFactor=1/255,nrMean=(0,0,0),RBSwap=True,scoreThres=0.4,nmsThres=0.4):
    blob = cv2.dnn.blobFromImage(image=img,
                                 scalefactor=scFactor,
                                 size=(416,416),
                                 mean=nrMean,
                                 swapRB=RBSwap,
                                 crop=False)
  
    imgHeight = img.shape[0]
    imgWidth = img.shape[1]

    net.setInput(blob)
    outLyrs = getOutputLayers(net)
    preds = net.forward(outLyrs)

    classId = []
    confidences = []
    boxes = []
    fboxes=[]
    fclasses=[]

    for scale in preds:
        for pred in scale:
            scores = pred[5:]
            clss = np.argmax(scores)
            confidence = scores[clss]
            
            if confidence > 0.4:
                xc = int(pred[0]*imgWidth)
                yc = int(pred[1]*imgHeight)
                w = int(pred[2]*imgWidth)
                h = int(pred[3]*imgHeight)
                x = xc - w/2
                y = yc - h/2

                classId.append(clss)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
      
    selected = cv2.dnn.NMSBoxes(bboxes=boxes,
                                scores=confidences,
                                score_threshold=scoreThres,
                                nms_threshold=nmsThres)
  
  # return empty list 
    if(len(selected) < 1):
        return [fboxes, fclasses]
  
    for j in selected[:,0]:
        fboxes.append(boxes[j])
        fclasses.append(classId[j])

    return [fboxes, fclasses]

In [9]:
def pltDetect(img, fboxes, fclasses, classes):
    colorset = np.random.uniform(0, 
                                 255,
                                 size=(len(classes), 3))
    txtlbl = ""
  
    for count, fbox in enumerate(fboxes):
        x = int(fbox[0])
        y = int(fbox[1])
        w = int(fbox[2])
        h = int(fbox[3])

        color  = colorset[fclasses[count]]
        txtlbl = str(classes[fclasses[count]])
        cv2.rectangle(img,
                      (x,y),
                      (x+w, y+h),
                      color,
                      2)

        cv2.putText(img,
                  txtlbl,
                  (x,y-5),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  0.5,
                  color,
                  1,
                  cv2.LINE_AA)

        cv2plt(img)

    # return number of cars detected
    return (txtlbl.count('car'))

  Using cached prophet-1.0.1.tar.gz (65 kB)
  Running setup.py clean for prophet
Failed to build prophet
    Running setup.py install for prophet: started
    Running setup.py install for prophet: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\yeewl\.conda\envs\ISS\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\yeewl\\AppData\\Local\\Temp\\pip-install-yxw8hzi4\\prophet_f3cb6f8ca28d43cb9970b8fc7c6b3a2c\\setup.py'"'"'; __file__='"'"'C:\\Users\\yeewl\\AppData\\Local\\Temp\\pip-install-yxw8hzi4\\prophet_f3cb6f8ca28d43cb9970b8fc7c6b3a2c\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\yeewl\AppData\Local\Temp\pip-wheel-b4wv0n5_'
       cwd: C:\Users\yeewl\AppData\Local\Temp\pip-install-yxw8hzi4\prophet_f3cb6f8ca28d43cb9970b8fc7c6b3a2c\
  Complete output (285 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib
  creating build\lib\prophet
  creating build\lib\prophet\stan_model
  Importing plotly failed. Interactive plots will

In [14]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = requests.get(url).content
    image = np.asarray(bytearray(resp), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    # return the image
    return image

In [15]:
def getVehicleCount(retrievalTime):
    url_traffic = 'https://api.data.gov.sg/v1/transport/traffic-images'

    params = {"date_time": retrievalTime.strftime("%Y-%m-%dT%H:%M:%S")}

    print(params)

    data = json.loads(requests.get(url_traffic, params=params).content)

    # Get image URL for the requested location
    try:
        url = [d.get('image', None) for d in data['items'][0]['cameras'] if d.get('location', None) == {'latitude': 1.357098686, 'longitude': 103.902042}][0]
    except:
        print("Unable to get data for {0}".format(retrievalTime))
        return -1 # Invalid Data

    image = url_to_image(url)
    filename = ".\\images\\" + retrievalTime.strftime("%Y-%m-%dT%H-%M-%S")+".jpg"
    cv2.imwrite(filename, image)
    #cv2plt(image)

    #[fboxes, fclasses] = yoloV3Detect(image)
    #pltDetect(image, fboxes, fclasses, classes)

    return (url)

**Nearest Weather Station will be 36 Kim Chuan Road**
Device ID: S43

In [16]:
def getRainfall(retrievalTime):
    url_weather = 'https://api.data.gov.sg/v1/environment/rainfall'
    params = {"date_time": retrievalTime.strftime("%Y-%m-%dT%H:%M:%S")} 

    data_weather = json.loads(requests.get(url_weather, params=params).content)

    # Get rainfall for the requested location
    try:
        rainfall = [d.get('value', None) for d in data_weather.get('items')[0].get('readings') if d.get('station_id', None) == "S43"][0]
    except:
        print("Unable to get data for {0}".format(retrievalTime))
        return -1 # Invalid Data

    return rainfall

In [24]:
# Declare Empty DataFrame
train_df = pd.DataFrame(columns=['Datetime', 'Vehicle Count', 'Rainfall'])

start = datetime.now().replace(hour=0, minute=0, second=0)

for dy in range(60, -1, -1):
    whichday = start - timedelta(days=dy)
    for hr in range(24):
        print("Iteration {} {}".format(dy, hr))
        print(whichday)
        now = whichday + timedelta(hours=hr)
        vehicleCount = getVehicleCount(now)
        rainfall = getRainfall(now)
        #print("Vehicle Count is {0}.".format(vehicleCount))
        print("Rainfall is {0} mm".format(rainfall))
        a_series = pd.Series([now, vehicleCount, rainfall], index = train_df.columns)
        train_df = train_df.append(a_series, ignore_index=True)
        train_df['Datetime'] = pd.to_datetime(train_df.Datetime,format='%d-%m-%Y %H:%M') 
    train_df.to_excel(".\DataGov.xlsx", index=False)

Iteration 60 0
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T00:00:00'}
Rainfall is 0 mm
Iteration 60 1
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T01:00:00'}
Rainfall is 0 mm
Iteration 60 2
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T02:00:00'}
Rainfall is 0 mm
Iteration 60 3
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T03:00:00'}
Rainfall is 0 mm
Iteration 60 4
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T04:00:00'}
Rainfall is 0 mm
Iteration 60 5
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T05:00:00'}
Rainfall is 0 mm
Iteration 60 6
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T06:00:00'}
Rainfall is 0 mm
Iteration 60 7
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T07:00:00'}
Rainfall is 0 mm
Iteration 60 8
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T08:00:00'}
Rainfall is 0 mm
Iteration 60 9
2021-12-31 00:00:00.532153
{'date_time': '2021-12-31T09:00:00'}
Rainfall is 0 mm
Iteration 60 10
2021-12-31 00:00:00.5321

Rainfall is 0 mm
Iteration 57 13
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T13:00:00'}
Rainfall is 0 mm
Iteration 57 14
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T14:00:00'}
Rainfall is 0 mm
Iteration 57 15
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T15:00:00'}
Rainfall is 0 mm
Iteration 57 16
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T16:00:00'}
Rainfall is 0 mm
Iteration 57 17
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T17:00:00'}
Rainfall is 0.2 mm
Iteration 57 18
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T18:00:00'}
Rainfall is 0 mm
Iteration 57 19
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T19:00:00'}
Rainfall is 0 mm
Iteration 57 20
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T20:00:00'}
Rainfall is 0 mm
Iteration 57 21
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T21:00:00'}
Rainfall is 0 mm
Iteration 57 22
2022-01-03 00:00:00.532153
{'date_time': '2022-01-03T22:00:00'}
Rainfall is 0 mm
Iteration 5

Rainfall is 0 mm
Iteration 53 2
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T02:00:00'}
Rainfall is 0 mm
Iteration 53 3
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T03:00:00'}
Rainfall is 0 mm
Iteration 53 4
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T04:00:00'}
Rainfall is 0 mm
Iteration 53 5
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T05:00:00'}
Rainfall is 0 mm
Iteration 53 6
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T06:00:00'}
Rainfall is 0 mm
Iteration 53 7
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T07:00:00'}
Rainfall is 0 mm
Iteration 53 8
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T08:00:00'}
Rainfall is 0 mm
Iteration 53 9
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T09:00:00'}
Rainfall is 0 mm
Iteration 53 10
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T10:00:00'}
Rainfall is 0 mm
Iteration 53 11
2022-01-07 00:00:00.532153
{'date_time': '2022-01-07T11:00:00'}
Rainfall is 0 mm
Iteration 53 12
2022-

Rainfall is 0 mm
Iteration 50 15
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T15:00:00'}
Rainfall is 0 mm
Iteration 50 16
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T16:00:00'}
Rainfall is 0 mm
Iteration 50 17
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T17:00:00'}
Rainfall is 0 mm
Iteration 50 18
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T18:00:00'}
Rainfall is 0 mm
Iteration 50 19
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T19:00:00'}
Rainfall is 0 mm
Iteration 50 20
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T20:00:00'}
Rainfall is 0 mm
Iteration 50 21
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T21:00:00'}
Rainfall is 0 mm
Iteration 50 22
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T22:00:00'}
Rainfall is 0 mm
Iteration 50 23
2022-01-10 00:00:00.532153
{'date_time': '2022-01-10T23:00:00'}
Rainfall is 0 mm
Iteration 49 0
2022-01-11 00:00:00.532153
{'date_time': '2022-01-11T00:00:00'}
Rainfall is 0 mm
Iteration 49 1

Rainfall is 0 mm
Iteration 46 4
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T04:00:00'}
Rainfall is 0 mm
Iteration 46 5
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T05:00:00'}
Rainfall is 0 mm
Iteration 46 6
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T06:00:00'}
Rainfall is 0 mm
Iteration 46 7
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T07:00:00'}
Rainfall is 0 mm
Iteration 46 8
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T08:00:00'}
Rainfall is 0 mm
Iteration 46 9
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T09:00:00'}
Rainfall is 0 mm
Iteration 46 10
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T10:00:00'}
Rainfall is 0 mm
Iteration 46 11
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T11:00:00'}
Rainfall is 0 mm
Iteration 46 12
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T12:00:00'}
Rainfall is 0 mm
Iteration 46 13
2022-01-14 00:00:00.532153
{'date_time': '2022-01-14T13:00:00'}
Rainfall is 0 mm
Iteration 46 14
202

...

Rainfall is 0 mm
Iteration 15 16
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T16:00:00'}
Rainfall is 0 mm
Iteration 15 17
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T17:00:00'}
Rainfall is 0 mm
Iteration 15 18
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T18:00:00'}
Rainfall is 0 mm
Iteration 15 19
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T19:00:00'}
Rainfall is 0 mm
Iteration 15 20
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T20:00:00'}
Rainfall is 0 mm
Iteration 15 21
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T21:00:00'}
Rainfall is 0 mm
Iteration 15 22
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T22:00:00'}
Rainfall is 0 mm
Iteration 15 23
2022-02-14 00:00:00.532153
{'date_time': '2022-02-14T23:00:00'}
Rainfall is 0 mm
Iteration 14 0
2022-02-15 00:00:00.532153
{'date_time': '2022-02-15T00:00:00'}
Rainfall is 0 mm
Iteration 14 1
2022-02-15 00:00:00.532153
{'date_time': '2022-02-15T01:00:00'}
Rainfall is 0 mm
Iteration 14 2


Rainfall is 0 mm
Iteration 11 5
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T05:00:00'}
Rainfall is 0 mm
Iteration 11 6
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T06:00:00'}
Rainfall is 0 mm
Iteration 11 7
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T07:00:00'}
Rainfall is 0 mm
Iteration 11 8
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T08:00:00'}
Rainfall is 0 mm
Iteration 11 9
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T09:00:00'}
Rainfall is 0 mm
Iteration 11 10
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T10:00:00'}
Rainfall is 0 mm
Iteration 11 11
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T11:00:00'}
Rainfall is 0 mm
Iteration 11 12
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T12:00:00'}
Rainfall is 0 mm
Iteration 11 13
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T13:00:00'}
Rainfall is 0 mm
Iteration 11 14
2022-02-18 00:00:00.532153
{'date_time': '2022-02-18T14:00:00'}
Rainfall is 0 mm
Iteration 11 15
20

Rainfall is 0 mm
Iteration 8 19
2022-02-21 00:00:00.532153
{'date_time': '2022-02-21T19:00:00'}
Rainfall is 0 mm
Iteration 8 20
2022-02-21 00:00:00.532153
{'date_time': '2022-02-21T20:00:00'}
Rainfall is 0 mm
Iteration 8 21
2022-02-21 00:00:00.532153
{'date_time': '2022-02-21T21:00:00'}
Rainfall is 0 mm
Iteration 8 22
2022-02-21 00:00:00.532153
{'date_time': '2022-02-21T22:00:00'}
Rainfall is 0 mm
Iteration 8 23
2022-02-21 00:00:00.532153
{'date_time': '2022-02-21T23:00:00'}
Rainfall is 0 mm
Iteration 7 0
2022-02-22 00:00:00.532153
{'date_time': '2022-02-22T00:00:00'}
Rainfall is 0 mm
Iteration 7 1
2022-02-22 00:00:00.532153
{'date_time': '2022-02-22T01:00:00'}
Rainfall is 0 mm
Iteration 7 2
2022-02-22 00:00:00.532153
{'date_time': '2022-02-22T02:00:00'}
Rainfall is 0 mm
Iteration 7 3
2022-02-22 00:00:00.532153
{'date_time': '2022-02-22T03:00:00'}
Rainfall is 0 mm
Iteration 7 4
2022-02-22 00:00:00.532153
{'date_time': '2022-02-22T04:00:00'}
Rainfall is 0 mm
Iteration 7 5
2022-02-22 00:

Rainfall is 0 mm
Iteration 4 9
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T09:00:00'}
Rainfall is 0 mm
Iteration 4 10
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T10:00:00'}
Rainfall is 0 mm
Iteration 4 11
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T11:00:00'}
Rainfall is 0 mm
Iteration 4 12
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T12:00:00'}
Rainfall is 0 mm
Iteration 4 13
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T13:00:00'}
Rainfall is 0 mm
Iteration 4 14
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T14:00:00'}
Rainfall is 0 mm
Iteration 4 15
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T15:00:00'}
Rainfall is 0 mm
Iteration 4 16
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T16:00:00'}
Rainfall is 0 mm
Iteration 4 17
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T17:00:00'}
Rainfall is 0 mm
Iteration 4 18
2022-02-25 00:00:00.532153
{'date_time': '2022-02-25T18:00:00'}
Rainfall is 0 mm
Iteration 4 19
2022-02-2

Rainfall is 0 mm
Iteration 1 23
2022-02-28 00:00:00.532153
{'date_time': '2022-02-28T23:00:00'}
Rainfall is 0 mm
Iteration 0 0
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T00:00:00'}
Rainfall is 0 mm
Iteration 0 1
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T01:00:00'}
Rainfall is 0 mm
Iteration 0 2
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T02:00:00'}
Rainfall is 0 mm
Iteration 0 3
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T03:00:00'}
Rainfall is 0 mm
Iteration 0 4
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T04:00:00'}
Rainfall is 0 mm
Iteration 0 5
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T05:00:00'}
Rainfall is 0 mm
Iteration 0 6
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T06:00:00'}
Rainfall is 0 mm
Iteration 0 7
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T07:00:00'}
Rainfall is 0 mm
Iteration 0 8
2022-03-01 00:00:00.532153
{'date_time': '2022-03-01T08:00:00'}
Rainfall is 0 mm
Iteration 0 9
2022-03-01 00:00:0

A day is considered to have “rained” if the total rainfall for that day is **0.2mm** or more.
[NEA Climate of Singapore](http://www.weather.gov.sg/climate-climate-of-singapore/)
